In [2]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd 

e:\AIO\AnamolyDetection_NLP_Time_Series\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [61]:
df = pd.read_excel("OnlineRetail\online_retail_II.xlsx", sheet_name="Year 2010-2011")
df_short = df[:6000]
df_short.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [62]:
embeddings = SentenceTransformer("all-MiniLM-L6-v2").encode(df_short['Description'])

In [63]:
print(len(embeddings[0]))

384


In [64]:
data = {
    "Sentence": df_short['Description'],
    ##            "Trunc_Sentence": df[text_col_name],
    "Embeddings": embeddings.tolist(),
    "TimeStamp": df_short['InvoiceDate']
}
df_embeddings = pd.DataFrame(data)
df_embeddings.head()

,Sentence,Embeddings,TimeStamp
0,WHITE HANGING HEART T-LIGHT HOLDER,"[0.016644464805722237, 0.1267160326242447, 0.0...",2010-12-01 08:26:00
1,WHITE METAL LANTERN,"[-0.027531709522008896, 0.09576564282178879, -...",2010-12-01 08:26:00
2,CREAM CUPID HEARTS COAT HANGER,"[-0.02481134794652462, 0.13718214631080627, 0....",2010-12-01 08:26:00
3,KNITTED UNION FLAG HOT WATER BOTTLE,"[-0.055034320801496506, 0.08257891982793808, -...",2010-12-01 08:26:00
4,RED WOOLLY HOTTIE WHITE HEART.,"[-0.05341784656047821, 0.012559731490910053, -...",2010-12-01 08:26:00


In [65]:
df_embeddings = df_embeddings[['TimeStamp', 'Embeddings']]
df_embeddings.head()

,TimeStamp,Embeddings
0,2010-12-01 08:26:00,"[0.016644464805722237, 0.1267160326242447, 0.0..."
1,2010-12-01 08:26:00,"[-0.027531709522008896, 0.09576564282178879, -..."
2,2010-12-01 08:26:00,"[-0.02481134794652462, 0.13718214631080627, 0...."
3,2010-12-01 08:26:00,"[-0.055034320801496506, 0.08257891982793808, -..."
4,2010-12-01 08:26:00,"[-0.05341784656047821, 0.012559731490910053, -..."


In [66]:
df_embeddings['TimeStamp'] = pd.to_datetime(df_embeddings['TimeStamp'])

In [68]:
train = embeddings[:5000]
test = embeddings[5000:]

In [73]:
import numpy as np
def to_sequences(x, y, seq_size=1):
    x_values = []
    y_values = []

    for i in range(len(x)-seq_size):
        #print(i)
        x_values.append(x[i:(i+seq_size)])
        y_values.append(y[i+seq_size])
        
    return np.array(x_values), np.array(y_values)

trainX, trainY = to_sequences(train, train, 30)
testX, testY = to_sequences(test, test, 30)

In [83]:
print(len(trainX[0]))
print(trainX.shape)

30
(4970, 30, 384)


In [114]:
from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential
from keras.layers import RepeatVector

# Encoder
autoencoder = Sequential()
autoencoder.add(LSTM(256, activation='relu', input_shape=(30,384), return_sequences = True))
# autoencoder.add(LSTM(128, activation='relu', return_sequences = True))
# autoencoder.add(LSTM(64, activation='relu', return_sequences = True))
autoencoder.add(LSTM(32, activation='relu', return_sequences = False))
autoencoder.add(Dropout(rate=0.2))

# Repeater
autoencoder.add(RepeatVector(30))

autoencoder.add(LSTM(32, activation='relu', return_sequences= True))
# autoencoder.add(LSTM(64, activation='relu', return_sequences = True))
# autoencoder.add(LSTM(128, activation='relu', return_sequences = True))
autoencoder.add(LSTM(256, activation='relu', return_sequences = True))
autoencoder.add(Dropout(rate=0.2))

autoencoder.add(TimeDistributed(Dense(384)))
autoencoder.compile(loss='mae', optimizer='adam')
autoencoder.summary()

Model: "sequential_65"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_204 (LSTM)                 │ (None, 30, 256)        │       656,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_205 (LSTM)                 │ (None, 32)             │        36,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_63 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector_25 (RepeatVector) │ (None, 30, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_206 (LSTM)                 │ (None, 30, 32)         │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_207 (LSTM)                 │ (None, 30, 256)        │       295,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_64 (Dropout)            │ (None, 30, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_25             │ (None, 30, 30)         │         7,710 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,005,342 (3.84 MB)

 Trainable params: 1,005,342 (3.84 MB)

 Non-trainable params: 0 (0.00 B)

In [115]:
autoencoder.fit(trainX, trainY, validation_split=0.2, epochs=10, batch_size=32)

Epoch 1/10


ValueError: Dimensions must be equal, but are 384 and 30 for '{{node compile_loss/mae/sub}} = Sub[T=DT_FLOAT](data_1, sequential_65_1/time_distributed_25_1/transpose_1)' with input shapes: [?,384], [?,30,30].

In [60]:
predicted_vectors = autoencoder.predict(embeddings)

16/16 ━━━━━━━━━━━━━━━━━━━━ 18s 906ms/step
